In [1]:
from Zenbot_demo import Zenbot
import uuid
import pandas as pd
import datetime
from Zenbot_demo import Zenbot
Zenbot_master = Zenbot(owner = {'name':000000}, events =[], tasks = [])


class Team(object):
	def __init__ (self, creator_id, zenbot_ids, description = ''):
		self.team_id = str(uuid.uuid4())
		self.creator = creator_id
		self.zenbot_ids = zenbot_ids
		self.description = description
		self.accessdict= {'level1':['Z3N', creator_id], 'level2':[], 'level3':[]}


	def report_tasks(self, caller_zenbotid, zenbotids_tocall = []):
		report = dict()
		if len(zenbotids_tocall) == 0:
			zenbotids_tocall= self.zenbot_ids
		if caller_zenbotid in self.accessdict['level1'] or caller_zenbotid in self.accessdict['level2']:
			for zenbotid in zenbotids_tocall:
				zenbot = Zenbot.zenbot_dict[zenbotid]
				report[zenbotid] = zenbot.report_tasks(caller_zenbotid)
		else:
			report = None

		return (report)

	#returns a dataframe of the tasks in the team
	def report_tasks_aggregate(self, caller_zenbotid, zenbotids_tocall = []):
		task_report = pd.DataFrame()
		if len(zenbotids_tocall) == 0:
			zenbotids_tocall= self.zenbot_ids

		if caller_zenbotid in self.accessdict['level1'] or caller_zenbotid in self.accessdict['level2'] or (caller_zenbotid in self.accessdict['level3'] and len(zenbotids_tocall) > 1):
			report = self.report_tasks(caller_zenbotid)
			
			#TODO change this
			n = 0
			for zenbotid in zenbotids_tocall:
				if n == 0:
					task_report = report[zenbotid]
					n += 1
				else:
					task_report = task_report.append(report[zenbotid])

			task_report.append(list(report.values()))
			#task_report.columns = ['title', 'project_id', 'category', 'progress', 'deadline', 'date_completed', 'dur_estim', 'time_on_task', 'effort', 'dependency_of', 'description', 'completed', 'overdue', 'at_risk']
			task_report = task_report.sort_values(by = 'deadline').reset_index(drop = True)
		else:
			task_report = None

		return (task_report)

	# returns a dict keys:zenbotids values:dicts keys:certifications, requiredcertifications values:dicts key:certification_name value:date_acquired/required
	def report_certificates(self, caller_zenbotid, zenbotids_tocall = []):
		certs = dict()
		if len(zenbotids_tocall) == 0:
			zenbotids_tocall= self.zenbot_ids
		if caller_zenbotid in self.accessdict['level1'] or caller_zenbotid in self.accessdict['level2'] or (caller_zenbotid in self.accessdict['level3'] and len(zenbotids_tocall) > 1):
			for zenbotid in zenbotids_tocall:
				zenbot = Zenbot.zenbot_dict[zenbotid]
				certs[zenbotid] = {'certifications': zenbot.report_certifications(caller_zenbotid), 'requiredcertifications': zenbot.report_requiredcertifications(caller_zenbotid)}

		return(certs)


	# returns a dict keys:zenbotids values:lists of events
	def report_events(self, caller_zenbotid, zenbotids_tocall = []):
		events = dict()
		if len(zenbotids_tocall) == 0:
			zenbotids_tocall= self.zenbot_ids
		if caller_zenbotid in self.accessdict['level1'] or caller_zenbotid in self.accessdict['level2'] or (caller_zenbotid in self.accessdict['level3'] and len(zenbotids_tocall) > 1):
			for zenbotid in zenbotids_tocall:
				zenbot = Zenbot.zenbot_dict[zenbotid]
				events[zenbotid] = zenbot.report_events(caller_zenbotid)

		return events

		return(certs)

	# returns a dict keys:zenbotids values:lists of trainings
	def report_trainings(self, caller_zenbotid, zenbotids_tocall = []):
		trainings = dict()
		if len(zenbotids_tocall) == 0:
			zenbotids_tocall= self.zenbot_ids
		if caller_zenbotid in self.accessdict['level1'] or caller_zenbotid in self.accessdict['level2'] or (caller_zenbotid in self.accessdict['level3'] and len(zenbotids_tocall) > 1):
			for zenbotid in zenbotids_tocall:
				zenbot = Zenbot.zenbot_dict[zenbotid]
				trainings[zenbotid] = zenbot.report_trainings()

		return (trainings)

	def report_by_individual(self, caller_zenbotid, zenbotids_tocall = []):
		report = dict()
		if len(zenbotids_tocall) == 0:
			zenbotids_tocall= self.zenbot_ids
		for zenbotid in zenbotids_tocall:
			zenbot = Zenbot.zenbot_dict[zenbotid]
			report[zenbotid] = zenbot.report_info(caller_zenbotid)

		return(report)

In [27]:
from numpy.random import randint, choice
from Task import Task

def get_random_cert() :
    cert = ''

    with open('training_courses.txt') as f :
        courses = f.readlines()
        while len(cert)<5 :  # sure its not an empty line
            cert = courses[randint(0,len(courses))]

    return cert

def task_generator():

    project_id = uuid.uuid4()
    deadline  = datetime.datetime.combine(datetime.date.today() + datetime.timedelta(days = randint(1,21)), datetime.time(0))
    creator = ''
    dur_estim = datetime.timedelta(hours=randint(1,4))
    importance = randint(1,4)
    effort = randint(1,6)

    available_categories = ["Congés", "Design", "Documentation", "Formations suivies", "Gestion des accès", "Other", "Project preliminary", "TC3/TC5", "Technical Implementation", "Training recieved", "Validation Execution", "WB", "WB (Gestion + Meetings)"]
    category = available_categories[randint(0,len(available_categories))]

    return Task(project_id, deadline, creator, dur_estim, importance, effort, category,  description= {})


def event_generator():
    event_type = choice(['training', 'meeting'],1, p =[0.3, 0.7])

    if event_type == 'training' :
        start = datetime.datetime.combine(datetime.date.today() + datetime.timedelta(days = randint(0,21)),datetime.time(hour=8))
        duration = datetime.timedelta(hours=3)
        end= start + duration
        loc = {}
        certification = get_random_cert()

        return Training(start, end, loc, certification )

    elif event_type == 'meeting' :
        start = datetime.datetime.combine(datetime.date.today() + datetime.timedelta(days = randint(0,21)),datetime.time(hour=randint(8,14)))
        duration = datetime.timedelta(hours=randint(1,3))
        end = start + duration

        loc = {}
        project_id = uuid.uuid4()
        participants = []

        return Meeting(start, end, loc, project_id, participants, tasks= [])

In [38]:
zen1 = Zenbot(owner = {'name':1000}, events =[], tasks = [])
zen2 = Zenbot(owner = {'name':2000}, events =[], tasks = [])
zen3 = Zenbot(owner = {'name':3000}, events =[], tasks = [])
zen4 = Zenbot(owner = {'name':4000}, events =[], tasks = [])
lesszen1 = Zenbot(owner = {'name':5000}, events =[], tasks = [])
admin = Zenbot(owner={'name':'Z3N'}, events=[], tasks= [])


In [39]:
admin.zenbotid = 'Z3N'
Zenbot.zenbot_dict['Z3N'] = admin
Zenbot.zenbot_dict[zen1.zenbotid] = zen1
Zenbot.zenbot_dict[zen2.zenbotid] = zen2
Zenbot.zenbot_dict[zen3.zenbotid] = zen3
Zenbot.zenbot_dict[zen4.zenbotid] = zen4
Zenbot.zenbot_dict[lesszen1.zenbotid] = lesszen1
Zenbot.zenbot_dict

{'1376dd7a-e9c4-4580-b3ee-41f2e7f56a76': <Zenbot_demo.Zenbot at 0x1083e2080>,
 '17a20207-9581-4131-bbfd-92ff3cc7f7fd': <Zenbot_demo.Zenbot at 0x1083d8860>,
 '221e61f0-c655-4e70-b640-04b978c9c7f4': 'None',
 '315562be-7d8d-45b9-9993-414ae77445e8': 'None',
 '38718337-9780-490b-89a5-6d4415508f3a': <Zenbot_demo.Zenbot at 0x1083e20f0>,
 '447945ae-c0b3-485b-8010-a5d04130269d': <Zenbot_demo.Zenbot at 0x1083cd400>,
 '473bc04c-c674-4d56-a950-ec43b261b61f': <Zenbot_demo.Zenbot at 0x1083d89e8>,
 '6a7a4528-d413-4202-bdd1-89c612513af8': <Zenbot_demo.Zenbot at 0x1083d87f0>,
 '7ae9302d-8989-489f-971f-6d836293b7a0': <Zenbot_demo.Zenbot at 0x1083d8940>,
 '7fbc9532-7a61-49db-a1be-2a39ed9ef93c': <Zenbot_demo.Zenbot at 0x1058e0da0>,
 '8c4676a9-b7e2-45c7-a0b5-561742c6bfd5': 'None',
 '8c9cd189-526d-4103-b649-d37d3321d932': <Zenbot_demo.Zenbot at 0x1083e2278>,
 '8d304a5f-3ef2-4172-adca-c2502441b838': 'None',
 '99abefa1-d39b-4a8e-8aa7-61eaf68498cb': <Zenbot_demo.Zenbot at 0x1083d89b0>,
 '9d923dd2-e903-4fa8-b70

In [4]:
for x in range(5):
    zen1.add_task(zen1.zenbotid, admin.zenbotid)
    zen2.add_task(zen2.zenbotid, admin.zenbotid)
    zen3.add_task(zen3.zenbotid, admin.zenbotid)
    zen4.add_task(zen4.zenbotid, admin.zenbotid)
    lesszen1.add_task(lesszen1.zenbotid, lesszen1.zenbotid)

Z3N
Z3N
Z3N
Z3N
a56abaee-6f5d-410e-b439-81608f9527de
Z3N
Z3N
Z3N
Z3N
a56abaee-6f5d-410e-b439-81608f9527de
Z3N
Z3N
Z3N
Z3N
a56abaee-6f5d-410e-b439-81608f9527de
Z3N
Z3N
Z3N
Z3N
a56abaee-6f5d-410e-b439-81608f9527de
Z3N
Z3N
Z3N
Z3N
a56abaee-6f5d-410e-b439-81608f9527de


In [40]:
for x in range(5):
    new_task = task_generator()
    zen2.add_task(zen2.zenbotid,   admin.zenbotid, new_task.project_id, new_task.deadline,
                 new_task.dur_estim, new_task.importance, new_task.effort, new_task.category,
                 description={'title': new_task.project_id, 'description': 'do this'})

  category :  WB
  created_by :  
  date_completed :  None
  deadline :  2018-01-27 00:00:00
  dependency_of :  []
  dependent_on :  []
  description :  {}
  dur_estim :  1:00:00
  effort :  1
  importance :  1
  progress :  0.0
  project_id :  430cc6f7-1958-4287-8885-e2295ce37e0a
  task_id :  0563b398-4510-4228-a8e9-eaa78650f08e
  time_on_task :  0:00:00
None
Z3N
  category :  Congés
  created_by :  
  date_completed :  None
  deadline :  2018-01-25 00:00:00
  dependency_of :  []
  dependent_on :  []
  description :  {}
  dur_estim :  3:00:00
  effort :  1
  importance :  3
  progress :  0.0
  project_id :  1f075353-07f1-4a21-92fb-171054b5b7fc
  task_id :  d995956e-97b1-4ea1-8954-a12f63c252e5
  time_on_task :  0:00:00
None
Z3N
  category :  Validation Execution
  created_by :  
  date_completed :  None
  deadline :  2018-01-29 00:00:00
  dependency_of :  []
  dependent_on :  []
  description :  {}
  dur_estim :  1:00:00
  effort :  5
  importance :  1
  progress :  0.0
  project_id : 

In [41]:
[t.prettify() for t in zen2.tasks]

  category :  WB
  created_by :  Z3N
  date_completed :  None
  deadline :  2018-01-27 00:00:00
  dependency_of :  []
  dependent_on :  []
  description :  {'title': UUID('430cc6f7-1958-4287-8885-e2295ce37e0a'), 'description': 'do this'}
  dur_estim :  1:00:00
  effort :  1
  importance :  1
  progress :  0.0
  project_id :  430cc6f7-1958-4287-8885-e2295ce37e0a
  task_id :  d4a13827-e13b-4ad0-8bd3-7d9591cf2a38
  time_on_task :  0:00:00
  category :  Congés
  created_by :  Z3N
  date_completed :  None
  deadline :  2018-01-25 00:00:00
  dependency_of :  []
  dependent_on :  []
  description :  {'title': UUID('1f075353-07f1-4a21-92fb-171054b5b7fc'), 'description': 'do this'}
  dur_estim :  3:00:00
  effort :  1
  importance :  3
  progress :  0.0
  project_id :  1f075353-07f1-4a21-92fb-171054b5b7fc
  task_id :  6e250d4e-be98-48e8-b2c6-13ea01297ab1
  time_on_task :  0:00:00
  category :  Validation Execution
  created_by :  Z3N
  date_completed :  None
  deadline :  2018-01-29 00:00:00
  

[None, None, None, None, None]

In [7]:
admin.grant_access(lesszen1.zenbotid, zen1.zenbotid, 'level2')
admin.grant_access(lesszen1.zenbotid, zen2.zenbotid, 'level2')
admin.grant_access(lesszen1.zenbotid, zen3.zenbotid, 'level2')
admin.grant_access(lesszen1.zenbotid, zen4.zenbotid, 'level2')

True
True
True
True


In [8]:
zen2.accessdict

{'level1': ['7fbc9532-7a61-49db-a1be-2a39ed9ef93c', 'Z3N'],
 'level2': ['a56abaee-6f5d-410e-b439-81608f9527de'],
 'level3': []}

In [9]:
zen2.report_info(zen1.zenbotid)

In [10]:
zen2.make_today_schedule()

In [42]:

zen1.certifications = dict([['cert1', datetime.datetime(2016,5,5)], ['cert2', datetime.datetime(2016,2,1)]])
zen1.requiredcertifications = dict([['cert1', datetime.datetime(2016,1,5)], ['cert2', datetime.datetime(2017,2,1)],['cert3', datetime.datetime(2017,5,1)]])

zen2.certifications = dict([['GESTION DES PERMIS', datetime.datetime(2016,5,5)], ['cert2', datetime.datetime(2016,2,1)]])
zen2.requiredcertifications = dict([['cert1', datetime.datetime(2016,1,5)], ['cert2', datetime.datetime(2017,2,1)],['cert3', datetime.datetime(2017,5,1)]])

zen3.certifications = dict([['Root Cause Analysis', datetime.datetime(2016,5,5)], ['cert2', datetime.datetime(2016,2,1)]])
zen3.requiredcertifications = dict([['cert1', datetime.datetime(2016,1,5)], ['cert2', datetime.datetime(2017,2,1)],['cert3', datetime.datetime(2017,5,1)]])

zen4.certifications = dict([['cert1', datetime.datetime(2016,5,5)], ['cert2', datetime.datetime(2016,2,1)]])
zen4.requiredcertifications = dict([['cert1', datetime.datetime(2016,1,5)], ['cert2', datetime.datetime(2017,2,1)],['cert3', datetime.datetime(2017,5,1)]])

In [12]:
zen1.new_trainings()
zen2.new_trainings()
zen3.new_trainings()
zen4.new_trainings()

2016-05-05 00:00:00
cert2
cert3
2016-05-05 00:00:00
cert2
cert3
2016-05-05 00:00:00
cert2
cert3
2016-05-05 00:00:00
cert2
cert3


In [13]:
zen2.tasks

In [14]:
team = Team(admin.zenbotid, [zen1.zenbotid, zen2.zenbotid, zen3.zenbotid, zen4.zenbotid])

In [15]:
team.report_tasks(admin.zenbotid)

{'7fbc9532-7a61-49db-a1be-2a39ed9ef93c':    title  project_id  category  progress                   deadline  \
 0  cert2      111111  training       0.0 2018-01-26 13:31:37.394108   
 1  cert3      111111  training       0.0 2018-01-26 13:31:37.394184   
 2  task1      123456     type1       0.0 2020-02-01 00:00:00.000000   
 3  task1      123456     type1       0.0 2020-02-01 00:00:00.000000   
 4  task1      123456     type1       0.0 2020-02-01 00:00:00.000000   
 5  task1      123456     type1       0.0 2020-02-01 00:00:00.000000   
 6  task1      123456     type1       0.0 2020-02-01 00:00:00.000000   
 
   date_completed       dur_estim time_on_task  effort dependency_of  \
 0           None 0 days 00:10:00       0 days       0                 
 1           None 0 days 00:10:00       0 days       0                 
 2           None 1 days 00:00:00       0 days       2                 
 3           None 1 days 00:00:00       0 days       2                 
 4           None 1 da

In [16]:
yourdict = {zen2.zenbotid : zen2.report_info(admin.zenbotid)}

In [17]:
yourdict

{'7fbc9532-7a61-49db-a1be-2a39ed9ef93c': {'certifications': {'cert1': datetime.datetime(2016, 5, 5, 0, 0),
   'cert2': datetime.datetime(2016, 2, 1, 0, 0)},
  'events': [],
  'requiredcertifications': {'cert1': datetime.datetime(2016, 1, 5, 0, 0),
   'cert2': datetime.datetime(2017, 2, 1, 0, 0),
   'cert3': datetime.datetime(2017, 5, 1, 0, 0)},
  'tasks':    title  project_id  category  progress                   deadline  \
  0  cert2      111111  training       0.0 2018-01-26 13:31:37.394108   
  1  cert3      111111  training       0.0 2018-01-26 13:31:37.394184   
  2  task1      123456     type1       0.0 2020-02-01 00:00:00.000000   
  3  task1      123456     type1       0.0 2020-02-01 00:00:00.000000   
  4  task1      123456     type1       0.0 2020-02-01 00:00:00.000000   
  5  task1      123456     type1       0.0 2020-02-01 00:00:00.000000   
  6  task1      123456     type1       0.0 2020-02-01 00:00:00.000000   
  
    date_completed       dur_estim time_on_task  effort d

In [18]:
from user_training_view_demo import output_training_dashboard

In [43]:
bot_test_id = zen2.zenbotid

output_training_dashboard(yourdict[bot_test_id])

KeyError: '17a20207-9581-4131-bbfd-92ff3cc7f7fd'

In [46]:
[(k,v) for k,v in yourdict[bot_test_id]['requiredcertifications'].items()]

[('cert1', datetime.datetime(2016, 1, 5, 0, 0)),
 ('cert2', datetime.datetime(2017, 2, 1, 0, 0)),
 ('cert3', datetime.datetime(2017, 5, 1, 0, 0))]